# Hands-On Large Language Models Language Understanding and Generation (Jay Alammar, Maarten Grootendorst)

## Chapter 1 -  An Introduction to Large Language Models

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map = "cuda",
    torch_dtype = "auto",
    trust_remote_code = True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`torch_dtype` is deprecated! Use `dtype` instead!


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [5]:
from transformers import pipeline

generator = pipeline("text-generation",
                     model = model,
                     tokenizer = tokenizer,
                     return_full_text = False,
                     max_new_tokens = 500,
                     do_sample = False)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [8]:
messages = [{"role" : "user", "content" : "Create a funny joke about chickens."}]

In [10]:
output = generator(messages, use_cache=False)
print(output[0]['generated_text'])

 Why did the chicken join the band? Because it had the drumsticks!


## Chapter 2 - Tokens & Embeddings

In [13]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

generation_output = model.generate(
input_ids=input_ids,
max_new_tokens=20,
use_cache = False
)

print(tokenizer.decode(generation_output[0]))

Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|> Subject: Sincere Apologies for the Gardening Mishap


Dear


In [19]:
print(input_ids[0])

tensor([14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
          293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
          372,  9559, 29889, 32001], device='cuda:0')


In [16]:
for id in input_ids[0]:
  print(tokenizer.decode(id))

Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.
Exp
lain
how
it
happened
.
<|assistant|>


In [20]:
print(generation_output[0])

tensor([14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
          293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
          372,  9559, 29889, 32001,  3323,   622, 29901,   317,  3742,   406,
         6225, 11763,   363,   278, 19906,   292,   341,   728,   481,    13,
           13,    13, 29928,   799], device='cuda:0')


In [21]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


In [22]:
text = """
English and CAPITALIZATION
🎵鸟
show_tokens False None elif == >= else: two tabs:" " Three tabs: "   "
12.0*50=600
"""

### Comparing different tokenizers

In [61]:
colors_list = [
'102;194;165', '252;141;98', '141;160;203',
'231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  token_ids = tokenizer(sentence).input_ids
  print(f'Number of tokens:' ,len(token_ids))

  for idx, t in enumerate(token_ids):
    print(f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
    tokenizer.decode(t) +
    '\x1b[0m',
    end=' '
  )


In [62]:
show_tokens(text, tokenizer_name = "google-bert/bert-base-uncased")

Number of tokens: 41
[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [63]:
show_tokens(text, tokenizer_name = "google-bert/bert-base-cased")

Number of tokens: 49
[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

In [64]:
show_tokens(text, tokenizer_name = "openai-community/gpt2")

Number of tokens: 48

 English  and  CAP ITAL IZ ATION 
 � � � � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"  "  Three  tabs :  "      " 
 12 . 0 * 50 = 600 
 

In [65]:
show_tokens(text, tokenizer_name = "google/flan-t5-xxl")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Number of tokens: 48
English and CA PI TAL IZ ATION  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

In [67]:
show_tokens(text, tokenizer_name = "facebook/galactica-1.3b")

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

Number of tokens: 56

 English  and  CAP ITAL IZATION 
 � � � � � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "   "  Three  t abs :   "     " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [66]:
show_tokens(text, tokenizer_name = "bigcode/starcoder2-15b")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

Number of tokens: 45

 English  and  CAPITAL IZATION 
 � � � � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"  "  Three  tabs :  "     " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [70]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base')
model = AutoModel.from_pretrained('microsoft/deberta-v3-small')

tokens =  tokenizer('Hello World', return_tensors = 'pt')

output = model(**tokens)[0]

In [71]:
output.shape

torch.Size([1, 4, 768])

In [73]:
for token in tokens['input_ids'][0]:
  print(tokenizer.decode(token))

[CLS]
Hello
 World
[SEP]


In [78]:
tokens = tokenizer(text, return_tensors = 'pt')
output = model(**tokens)

In [79]:
output

BaseModelOutput(last_hidden_state=tensor([[[-0.0449, -0.0384, -0.0019,  ..., -0.0492, -0.0389, -0.0698],
         [ 0.4824,  1.1953,  0.0209,  ..., -0.6417,  1.3388, -0.4586],
         [-0.0498,  0.0096,  0.3172,  ...,  0.1513,  0.2044,  0.0811],
         ...,
         [ 0.5234, -0.7091, -0.0033,  ..., -0.4279,  0.2046, -0.0341],
         [ 1.0002, -0.4685,  0.2729,  ...,  0.3513, -0.1965, -0.2895],
         [-0.0607, -0.0443, -0.0137,  ..., -0.0374, -0.0401, -0.0737]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [80]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
vector = model.encode("Best movie ever!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [81]:
vector.shape

(768,)

In [83]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 19.6 MB/s eta 0:00:00


In [84]:
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [85]:
model.most_similar([model['king']], topn=11)

[('king', 1.0000001192092896),
 ('prince', 0.8236179351806641),
 ('queen', 0.7839043140411377),
 ('ii', 0.7746230363845825),
 ('emperor', 0.7736247777938843),
 ('son', 0.766719400882721),
 ('uncle', 0.7627150416374207),
 ('kingdom', 0.7542161345481873),
 ('throne', 0.7539914846420288),
 ('brother', 0.7492411136627197),
 ('ruler', 0.7434253692626953)]

### Making a song recommendation system using word embeddings concepts

In [86]:
import pandas as pd
from urllib import request

data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')
lines = data.read().decode("utf-8").split('\n')[2:]
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [87]:
songs_df

,title,artist
id,,
0,Gucci Time (w\/ Swizz Beatz),Gucci Mane
1,Aston Martin Music (w\/ Drake & Chrisette Mich...,Rick Ross
2,Get Back Up (w\/ Chris Brown),T.I.
3,Hot Toddy (w\/ Jay-Z & Ester Dean),Usher
4,Whip My Hair,Willow
...,...,...
75258,USA Today,Alan Jackson
75259,Superstar,Raul Malo
75260,Romancin' The Blues,Giacomo Gates


In [88]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

Playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

Playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

In [89]:
from gensim.models import Word2Vec
# Train our Word2Vec model
model = Word2Vec(
playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [90]:
song_id = 2172
model.wv.most_similar(positive=str(song_id))

[('1922', 0.9986104369163513),
 ('6641', 0.9963059425354004),
 ('3119', 0.9956034421920776),
 ('10105', 0.9953237175941467),
 ('11517', 0.9950727820396423),
 ('6685', 0.9947185516357422),
 ('2849', 0.9947027564048767),
 ('5586', 0.9945303797721863),
 ('3116', 0.9938734173774719),
 ('20010', 0.9937389492988586)]

In [91]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [92]:
import numpy as np
def print_recommendations(song_id):
  similar_songs = np.array(
    model.wv.most_similar(positive=str(song_id),topn=5))[:,0]
  return  songs_df.iloc[similar_songs]
print_recommendations(2172)

,title,artist
id,,
1922,One,Metallica
6641,Shout At The Devil,Motley Crue
3119,There's Only One Way To Rock,Sammy Hagar
10105,Three Lock Box,Sammy Hagar
11517,Mary Had A Little Lamb,Stevie Ray Vaughan & Double Trouble
